[View in Colaboratory](https://colab.research.google.com/github/schwaaweb/aimlds1_07-TheMachineLearningFramework/blob/master/M07_A--Model_Tuning_Assignment.ipynb)

# Part One: K-fold Cross Validation

The challenge of training machine learning models is to be able to make accurate predictions on previously unseen real-world data in spite of the fact that we only have a finite training dataset to learn from. 

One way of validating our model's quality-of-fit and avoiding overfitting/underfitting, is to use the test_train_split method like we did in the code challenge. With this method, the randomly selected test dataset can be used to evaluate how our model performs on data that it has not yet seen in the training process. However, there are downsides to this approach:

*   We lose a valuable portion of data that we would prefer to be able to train on to serve as the test dataset. We would prefer to have both the testing and training datasets be as large as possible.
*   With small datasets, measures of our model's quality using the test_train_split method often have a high variance. (We saw this behavior when we changed the random seed in the code challenge)

We can reduce the severity of both of these drawbacks by using what is called K-fold Cross Validation:

[Short Video Explaining K-Fold Cross Validation](https://www.youtube.com/watch?v=TIgfjmp-4BA)

[How to Implement K-Fold Cross Validation on the Pima Indians Diabetes dataset](https://machinelearningmastery.com/evaluate-performance-machine-learning-algorithms-python-using-resampling/)

## DO THIS:

**1)** Train a logistic regression model on the titanic dataset predicting survivors first using a 20-80% test_train_split and print the accuracy of your model using 5 different random seeds.

**2)** Use 5-fold Cross Validation on the titanic dataset. Print out the accuracies from each of the 5 folds of the cross validation, then print the final mean and standard deviation of those cross validation accuracies. How do the accuracies on each of the inidvidual folds compare to the accuracies of the test_train_split approach? Is the variance in accuracies of the cross-validation approach higher or lower than the variance of the test_train_split approach? 

**3)** Try using 3-fold Cross Validation as well as 10-fold cross validation. How does the number of folds in the cross-validation process affect the outcome? How many folds should be used?

---
I would give you more boilerplate code here, but I don't want to make it too easy. The articles linked above should be sufficient for this purpose.

In [1]:
##### YOUR CODE HERE ##### - Feel free to add code cells as necessary.
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

titanic = sns.load_dataset('titanic')

# drop duplicate/analogous columns
titanic = titanic.drop(['alive',
                        'adult_male',
                        'who',
                        'class',
                        'embark_town'], axis=1)

# take care of missing data
titanic['embarked'] = titanic['embarked'].fillna(method='ffill')
titanic = titanic.drop(['deck'], axis=1)
titanic['age'] = titanic['age'].fillna(method='ffill')

# convert binomials and categoricals to encoded labels
for label in ['embarked', 'sex', 'alone']:
    titanic[label] = LabelEncoder().fit_transform(titanic[label])

titanic.head()




,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,1,22.0,1,0,7.2500,2,0
1,1,1,0,38.0,1,0,71.2833,0,0
2,1,3,0,26.0,0,0,7.9250,2,1
3,1,1,0,35.0,1,0,53.1000,2,0
4,0,3,1,35.0,0,0,8.0500,2,1


In [2]:
Y = titanic[['survived']]
X = titanic.drop(['survived'], axis=1)
print(Y.head())
print(X.head())
print(Y.shape)
print(X.shape)
print(titanic.shape)

   survived
0         0
1         1
2         1
3         1
4         0
   pclass  sex   age  sibsp  parch     fare  embarked  alone
0       3    1  22.0      1      0   7.2500         2      0
1       1    0  38.0      1      0  71.2833         0      0
2       3    0  26.0      0      0   7.9250         2      1
3       1    0  35.0      1      0  53.1000         2      0
4       3    1  35.0      0      0   8.0500         2      1
(891, 1)
(891, 8)
(891, 9)


**1)** Train a logistic regression model on the titanic dataset predicting survivors first using a 20-80% test_train_split and print the accuracy of your model using 5 different random seeds.


In [3]:
#import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression



seeds = np.random.randint(0, 999, 5)
test_size = 0.2 

print('random\nseed\tAccuracy\n')
for seed in seeds:
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)

    model = LogisticRegression()

    model.fit(X_train, np.ravel(Y_train))
    result = model.score(X_test, Y_test)
    #f"{seed} == seed | Accuracy: {result*100.0:.3f}" #  :.3f limits the decimal places to 3
    print(seed, '\t', result*100.0)
    

random
seed	Accuracy

111 	 79.3296089385475
657 	 79.3296089385475
874 	 81.56424581005587
420 	 81.00558659217877
679 	 78.2122905027933


**2)** Use 5-fold Cross Validation on the titanic dataset. Print out the accuracies from each of the 5 folds of the cross validation, then print the final mean and standard deviation of those cross validation accuracies. How do the accuracies on each of the inidvidual folds compare to the accuracies of the test_train_split approach? Is the variance in accuracies of the cross-validation approach higher or lower than the variance of the test_train_split approach? 


In [4]:
# Evaluate using Cross Validation

from sklearn.linear_model import LogisticRegression

print('\n 5-fold Cross Validation')
print('random seeds: ', seeds, '\n')

for seed in seeds:
    num_instances = len(X)

    kfold = model_selection.KFold(n_splits=5, random_state=seed)

    model = LogisticRegression()
    results = model_selection.cross_val_score(model, X, np.ravel(Y), cv=kfold)
    #f"Accuracy: {results.mean()*100.0:.3f} ({results.std()*100.0:.3f})"
    print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))



 5-fold Cross Validation
random seeds:  [111 657 874 420 679] 

Accuracy: 79.126% (2.707%)
Accuracy: 79.126% (2.707%)
Accuracy: 79.126% (2.707%)
Accuracy: 79.126% (2.707%)
Accuracy: 79.126% (2.707%)


### It appears that the 20% Test 80% Train test set is more accurate than the 5 Fold Cross Validation

**3)** Try using 3-fold Cross Validation as well as 10-fold cross validation. How does the number of folds in the cross-validation process affect the outcome? How many folds should be used?


In [5]:
print('\n 3-fold Cross Validation')
print('random seeds: ', seeds, '\n')

for seed in seeds:
    num_instances = len(X)

    kfold = model_selection.KFold(n_splits=3, random_state=seed)

    model = LogisticRegression()
    results = model_selection.cross_val_score(model, X, np.ravel(Y), cv=kfold)
    #f"Accuracy: {results.mean()*100.0:.3f} ({results.std()*100.0:.3f})"
    print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))



 3-fold Cross Validation
random seeds:  [111 657 874 420 679] 

Accuracy: 78.900% (0.420%)
Accuracy: 78.900% (0.420%)
Accuracy: 78.900% (0.420%)
Accuracy: 78.900% (0.420%)
Accuracy: 78.900% (0.420%)


In [6]:
print('\n 10-fold Cross Validation')
print('random seeds: ', seeds, '\n')

for seed in seeds:
    num_instances = len(X)

    kfold = model_selection.KFold(n_splits=10, random_state=seed)

    model = LogisticRegression()
    results = model_selection.cross_val_score(model, X, np.ravel(Y), cv=kfold)
    #f"Accuracy: {results.mean()*100.0:.3f} ({results.std()*100.0:.3f})"
    print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))



 10-fold Cross Validation
random seeds:  [111 657 874 420 679] 

Accuracy: 79.464% (3.497%)
Accuracy: 79.464% (3.497%)
Accuracy: 79.464% (3.497%)
Accuracy: 79.464% (3.497%)
Accuracy: 79.464% (3.497%)


### The standard deviation is lowest with the 3-fold Cross Validation while the accuracy his highest with the 10 fold cross validation but it is less then 1% more accurate while the standard deviation is 3% lower with the 3-fold cross validation. 

### I would go with the 3-fold cross validation because of the smaller standard deviation

# Part Two: Hyperparameter Tuning

An important technique for improving the accuracy of a machine learning model is to undertake a process known as Hyperparameter Tuning or Hyperparameter Optimization. In order to understand this process, we first need to understand the difference between a model parameter and a model hyperparameter. 

### What is a model parameter?

A model parameter is a value that is generated by fitting our model to training data and is key to generating predictions with that model. They are **internal** to our model and we often are trying to estimate them as best as possible when we train the algorithm. 

For example, the parameters of a linear regression model would be its intercept value as well as the coefficient values on each of the X variables. Estimates of these crucial values (parameters) are obtained by fitting to the training data, perfectly define the model, are internal to the model, and are key to generating predictions. They are model parameters in every sense. 

### What is a model hyperparameter?

Hyperparameters are values that are key to how well our algorithm runs, yet are **external** to our model and cannot be estimated from the training process. They are more like settings for our algorithm which must be designated before it is run and impact its performance. Here is some further reading:

[Hyperparamters explanation on Quora](https://www.quora.com/What-are-hyperparameters-in-machine-learning)

[Jason Brownlee Article on the difference between Parameters and Hyperparameters](https://machinelearningmastery.com/difference-between-a-parameter-and-a-hyperparameter/)

### How do we find the best hyperparameters?

Since we can't learn the best hyperparameters for our model from the data, we essentially just pick values and see which ones lead to the highest accuracy. This can be a tedious and complex process especially for certain models like neural networks which can have dozens of hyperparameters. We will get you familiar with the process using a more simple logistic regression model. 

### How do you know what hyperparameters exist for your particular model? 

Most models/libraries have default hyperparameters that will be used if we don't specify them. In the model selection process you might try out multiple models on a dataset and see which one gets you the highest out-of-the-box performance, (using the default hyperparameters) and then pick a couple of the highest performing algorithms and attempt hyperparameter tuning on them to compare how different models benefit from this process. Once you have narrowed down the models that you would like to tune, a quick google search can tell you what hyperparameters exist for that algorithm. 

Often you can learn about potential hyperparameters by looking at the documentation for a given algorithm in a library, here's the documentation for sklearn's logistic regression, see if you can spot the hyperparameters:

[scikit-learn logistic regression docs](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)



## DO THIS: 

Lets hyperparameter tune our **titanic** predictions using 5-fold cross validation to compare the accuracy of our tuned models. 

### Manual Hyperparameter Tuning:

For our assignment today we are going to tune the 'C value' also known as the 'regularization strength' of our logistic regression as well as 'penalty' of our logistic regression algorithm.

Read up on the regularlization strength and penalty of a logistic regression function. What might be some good values to test out? Hint: Look at the parameter definitions on the sci-kit learn logistic regression documentation. 

[scikit-learn logistic regression docs](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)

[Regularization in Logistic Regression](https://www.kdnuggets.com/2016/06/regularization-logistic-regression.html)

Fit your model 5 different times using 5 different C values of your choosing. Which value gives the highest accuracy? 

There are only two penalty values that we can use. Evaluate the model two more times using each penalty once. Which penalty gives the highest accuracy?

In [7]:
# The sample code below uses the Pima Indans Diabetes Dataset. 
# Here we are setting the C value hyperparameter to 1 and the penalty hyperparameter to "l1". 
# You can designate your hyperparameters in a similar fashion.

import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
Xd = array[:,0:8]
Yd = array[:,8]
num_instances = len(Xd)
seed = 7
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model = LogisticRegression(C=1, penalty='l1') ##### This is the important line
results = model_selection.cross_val_score(model, Xd, Yd, cv=kfold)
print(results)

[0.75974026 0.72727273 0.76623377 0.83006536 0.76470588]


In [8]:
print('\n 5-fold Cross Validation')
#print('random seeds: ', seeds, '\n')

cz = [0.01,0.1,1,10,100]

for c in cz:
    num_instances = len(X)

    kfold = model_selection.KFold(n_splits=5, random_state=7)

    model = LogisticRegression(C=c, penalty='l1')
    results = model_selection.cross_val_score(model, X, np.ravel(Y), cv=kfold)
    #f"Accuracy: {results.mean()*100.0:.3f} ({results.std()*100.0:.3f})"
    #print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))
    print(results)


 5-fold Cross Validation
[0.62569832 0.70786517 0.64044944 0.66292135 0.70786517]
[0.79888268 0.80337079 0.7752809  0.74719101 0.82022472]
[0.78212291 0.78651685 0.78651685 0.75842697 0.83146067]
[0.78212291 0.78089888 0.78651685 0.76404494 0.83707865]
[0.78212291 0.7752809  0.78651685 0.76404494 0.84269663]


In [9]:
print('\n 5-fold Cross Validation')
#print('random seeds: ', seeds, '\n')

cz = [0.01,0.1,1,10,100]

for c in cz:
    #num_instances = len(X)

    kfold = model_selection.KFold(n_splits=5, random_state=7)

    model = LogisticRegression(C=c, penalty='l2')
    results = model_selection.cross_val_score(model, X, np.ravel(Y), cv=kfold)
    #f"Accuracy: {results.mean()*100.0:.3f} ({results.std()*100.0:.3f})"
    #print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))
    print(results)


 5-fold Cross Validation
[0.68156425 0.70224719 0.68539326 0.69101124 0.76966292]
[0.79888268 0.80337079 0.76966292 0.75842697 0.83707865]
[0.78212291 0.80337079 0.7752809  0.75842697 0.83707865]
[0.78212291 0.78089888 0.78651685 0.76404494 0.83146067]
[0.78212291 0.78089888 0.78651685 0.76404494 0.84269663]


### 1) Grid-Search Hyperparameter Tuning:

Imagine that your algorithm has 12 different potential hyperparameters and each them can take on 5 different values. Lets say that it takes your laptop 4 seconds to fit each fold of cross validation. For each 5-fold cross-validation it would then take 20 seconds to fit your model and get an accuracy reading reported back. Now imagine that you want to test every possible combination of hyperparameters on your algorithm to get the absolute highest accuracy. You can see how this might become exceedingly tedious and time-consuming to perform by hand. Some hyperparameters (like the C value) have much more than 5 potential values, making hyperparameter tuning a huge task. 

It is for this reason that more advanced optimization techniques exist, one of which we will be exploring today called GridSearch.

### What does GridSearch do?

GridSearch takes a dictionary of all of the different hyperparameters that you want to test, and then feeds all of the different combinations through the algorithm for you and then reports back to you which one had the highest accuracy. Pretty slick right? 

Here is some boilerplate code you can reference to create your implementations:

[Chris Albon Logistic Regression sklearn Hyperparameter Tuning with GridSearch](https://chrisalbon.com/machine_learning/model_selection/hyperparameter_tuning_using_grid_search/)

In [10]:
# These import statements might be useful to you. 
import numpy as np
from sklearn.model_selection import GridSearchCV

In [11]:
# Create logistic regression object
kfold = model_selection.KFold(n_splits=5, random_state=7)
model = LogisticRegression()

In [12]:
# Create a list of all of the different penalty values that you want to test and save them to a variable called 'penalty'
penalty = ['l1', 'l2']


In [13]:
# Create a list of all of the different C values that you want to test and save them to a variable called 'C'

C = np.logspace(1, 10, 5)


In [14]:
# Now that you have two lists each holding the different values that you want test, use the dict() function to combine them into a dictionary. 
# Save your new dictionary to the variable 'hyperparameters'
# Print out the dictionary if you're curious as to what it euds up looking like.
hyperparameters = dict(C=C, penalty=penalty)

In [15]:
# Fit your model using gridsearch
#clf = GridSearchCV(model, hyperparameters, cv=kfold, verbose=0)
clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)


In [16]:
best_model = clf.fit(X, np.ravel(Y))

In [19]:
# Print the best penalty and C value from best_model.best_estimator_.get_params()

#print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best C: 10.0


In [20]:
# Print out all of the different combinations of your grid search values and their corresponding accuracies.
# https://stackoverflow.com/questions/22155953/how-to-print-out-an-accuracy-score-for-each-combination-within-gridsearch
best_model.predict(X)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,

In [22]:
from pprint import pprint
pprint(clf.cv_results_)

{'mean_fit_time': array([0.01547661, 0.00605311, 0.01077943, 0.0060544 , 0.01046462,
       0.00655975, 0.01174746, 0.00656991, 0.01036062, 0.00626678]),
 'mean_score_time': array([0.00073004, 0.00059633, 0.00059767, 0.00051832, 0.00062013,
       0.00057187, 0.00065579, 0.0005619 , 0.00062051, 0.00057712]),
 'mean_test_score': array([0.78787879, 0.78787879, 0.78787879, 0.78787879, 0.78787879,
       0.78787879, 0.78787879, 0.78787879, 0.78787879, 0.78787879]),
 'mean_train_score': array([0.80275525, 0.80219188, 0.80303575, 0.80303575, 0.80303575,
       0.80303575, 0.80303575, 0.80303575, 0.80303575, 0.80303575]),
 'param_C': masked_array(data=[10.0, 10.0, 1778.2794100389228, 1778.2794100389228,
                   316227.7660168379, 316227.7660168379,
                   56234132.51903491, 56234132.51903491, 10000000000.0,
                   10000000000.0],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?'

What hyperparameters give you the highest accuracy? Keep on testing diferent values and report the hyperparameters that give you the highest accuracy.

# Stretch Goals:

Explore more advanced automated approaches to hyperparameter tuning. Try and implemenet a random search approach: 

[Random Search Hyperparameter Tuning](https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/)

Then try a Bayesian Optimization Approach:

[Bayesian Optimization](https://thuijskens.github.io/2016/12/29/bayesian-optimisation/)

[scikit-optimize](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

[optunity](http://optunity.readthedocs.io/en/latest/notebooks/notebooks/sklearn-automated-classification.html)

You could also try writing a blog post to show how well you understand Cross Validation or Hyperparameter Tuning, both are key concepts to practicing machine learning and would be valuable to demonstrate proficency in.
